<a href="https://colab.research.google.com/github/cedric-bit/Geospatial-Data-Science-2023/blob/main/loadDATA509.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install gtfs-realtime-bindings


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
!apt-get install python3.7
!apt-get install python3.7-venv





Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3.7 is already the newest version (3.7.16-1+focal1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3.7-venv is already the newest version (3.7.16-1+focal1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [10]:
!pip install --upgrade protobuf



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
from google.colab import drive
drive.mount('/content/drive')
import os
from datetime import datetime

shortcut_path = '/content/drive/MyDrive/Geospatial Data Science 2023' 
geospacialdata_folder_name = "Geospatial Data Science 2023"

# Vérifiez si le nom du dossier est présent dans le chemin du raccourci
if geospacialdata_folder_name in shortcut_path:
    base_path = shortcut_path
else:
    for folder in os.listdir(shortcut_path):
        if folder == geospacialdata_folder_name:
            base_path = os.path.join(shortcut_path, geospacialdata_folder_name)
            break

if base_path is not None:
    print(f"Dossier geospacialdata trouvé: {base_path}")
else:
    print("Dossier geospacialdata non trouvé")

# Liste les fichiers et les dossiers dans le répertoire Geospatial Data Science 2023
#for item in os.listdir(base_path):
 #   item_path = os.path.join(base_path, item)
  #  if os.path.isdir(item_path):
   #     print(f"Dossier: {item}")

   
# Sélectionnez le dossier 2023-03-20
selected_folder = "2023-03-20"
selected_folder_path = os.path.join(base_path, selected_folder)

# Affichez le contenu du dossier sélectionné
for item in os.listdir(selected_folder_path):
    item_path = os.path.join(selected_folder_path, item)
    if os.path.isdir(item_path):
        print(f"Dossier: {item}")
  #  elif os.path.isfile(item_path):
   #     print(f"Fichier: {item}")




def check_file_format(file_path):
    with open(file_path, 'rb') as f:
        signature = f.read(2)

    if signature == b'\x1f\x8b':
        print(f"{file_path} est un fichier gzip.")
    else:
        print(f"{file_path} n'est pas un fichier gzip.")

# Utilisez cette fonction pour vérifier le format d'un fichier .gtfsrt spécifique
for gtfsrt_file in os.listdir(selected_folder_path):
    gtfsrt_file_path = os.path.join(selected_folder_path, gtfsrt_file)
    #check_file_format(gtfsrt_file_path)

#from google.transit import gtfs_realtime_pb2



for gtfsrt_file in os.listdir(selected_folder_path):
    if gtfsrt_file.endswith('.gtfsrt'):
        gtfsrt_file_path = os.path.join(selected_folder_path, gtfsrt_file)
       # process_gtfsrt_file(gtfsrt_file_path)


import osmnx as ox


from google.transit import gtfs_realtime_pb2 as gtfs_realtime
from google.protobuf import text_format


def process_gtfsrt_file(file_path):
    feed = gtfs_realtime.FeedMessage()
    
    with open(file_path, 'rb') as f:
        content = f.read()
        feed.ParseFromString(content)

    return feed

feeds = []

for gtfsrt_file in os.listdir(selected_folder_path):
    if gtfsrt_file.endswith('.gtfsrt'):
        gtfsrt_file_path = os.path.join(selected_folder_path, gtfsrt_file)
        try:
            feed = process_gtfsrt_file(gtfsrt_file_path)
            feeds.append(feed)
            print(f"Processed {gtfsrt_file_path}")
        except Exception as e:
            print(f"Failed to process {gtfsrt_file_path}: {e}")






Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dossier geospacialdata trouvé: /content/drive/MyDrive/Geospatial Data Science 2023
Processed /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-20/1679322121.gtfsrt
Processed /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-20/1679322154.gtfsrt
Processed /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-20/1679322182.gtfsrt
Processed /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-20/1679322215.gtfsrt
Processed /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-20/1679322241.gtfsrt
Processed /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-20/1679322274.gtfsrt
Processed /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-20/1679322302.gtfsrt
Processed /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-20/1679322335.gtfsrt
Processed /content/drive/MyDrive/Geospatial Data Science 202